<div style="background-color: #bfd630; font-family: Calibri, sans-serif; padding: 20px;">



   <div style="text-align: center;">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaJWG7PzF3toxaRMB1-JicpqMgJuEXATd0fg&" style="width: 120px; margin-top: 20px; margin-bottom: 60px;">
   </div>

   <div style="text-align: center; font-size: 24px; font-weight: bold; font-family: Calibri; color: #000000; margin-bottom: 10px;">
      Machine Learning Project | To Grant or Not to Grant
   </div>
   
   <div style="text-align: center; font-family: Calibri; font-size: 22px; color: #000000; font-weight: bold; margin-bottom: 20px;">
      4. N Stage Learning Model
   </div>

   <div style="text-align: center; font-size: 18px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 5px;">
      Nova Information Management School
   </div>

   <div style="text-align: center; font-size: 18px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 20px;">
      Universidade Nova de Lisboa
   </div>
        <div style="text-align: center; font-size: 16px; font-family: Calibri; font-weight: bold; color: #333333; margin-bottom: 10px;">
      Master in Data Science and Advanced Analytics
   </div>
 
   <div style="text-align: center;">
      <img src="https://cdn.prod.website-files.com/617accb8b04ef2b3feffa61b/6581e90d485a9976c3576a46_how-does-workers-comp-work.jpg" style="width: 350px; margin-top: 20px; margin-bottom: 60px;">
   </div>




   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; font-weight: bold; margin-bottom: 20px;">
      Project Group: 32
   </div>

   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 40px">
      Klimentina Gilevska -  20240747 <br>
      Maria Assunção -  20211605 <br>
      Rita Matos -  20211642 <br>
      Rita Wang -  20240551 <br>
      Sven Goerdes -  20240503 <br>
   </div>

   <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 10px">
      Fall/Spring Semester 2024-2025
   </div>

  <div style="text-align: center; font-family: Calibri; font-size: 16px; color: #333333; margin-bottom: 20px;">
      22th November 2024
   </div>

   
</div>

<span style="color:red"><b><<<<<<< local</b></span>

<span style="color:red"><b>=======</b></span>

<a id = "toc"></a>
## Table of Contents

* __[1. Importing Libraries & Dataset](#importing-libraries-dataset)__
    * [1.1. Import Necessary Libraries](#import-necessary-libraries)
    * [1.2. Load the Dataset](#load-the-dataset)
* __[2. Prepare the Dataset](#prepare-the-dataset)__
    * [2.1. Encode the Target Variable](#encode-the-target-variable)
    * [2.2. Non-encoded Target Variable as Series](#non-encoded-target-variable-as-series)
    * [2.3. Separate the Variables into Numerical and Categorical](#separate-the-variables-into-numerical-and-categorical)
* __[3. Data Scaling](#data-scaling)__
    * [3.1. Normalization](#normalization)
    * [3.2. Standardization](#standardization)
* __[4. Feature Selection](#feature-selection)__
    * [4.1. Categorical: Filter Methods](#categorical-filter-methods)
        * [4.1.1. Chi square and Cramer's V](#chi-square-and-cramers-v)
        * [4.1.2. Mutual Information](#mutual-information)
        * [4.1.3. Filtered Categoricals](#filtered-categoricals)
    * [4.2. Numerical: Filter Methods](#numerical-filter-methods)
        * [4.2.1. Univariate Variance](#univariate-variance)
        * [4.2.2. Spearman Correlation](#spearman-correlation)
        * [4.2.3. Mutual Information](#mutual-information-2)
        * [4.2.4. Filtered Numerical](#filtered-numerical)
    * [4.3. Combine the Filtered Datasets](#combine-the-filtered-datasets)
    * [4.4. Feature Selection on the Combined Dataset](#feature-selection-on-the-combined-dataset)
        * [4.4.1. Random Forest Important Features](#random-forest-important-features)
        * [4.4.2. XGBoost Important Features](#xgboost-important-features)
        * [4.4.3. Lasso](#lasso)
        * [4.4.4. Ridge](#ridge)
        * [4.4.5. Vote for the Best Features](#vote-for-the-best-features)
* __[5. Find the Best Model](#find-the-best-model)__
* __[6. Deal with Imbalanced Dataset](#deal-with-imbalanced-dataset)__
* __[7. Final Model](#final-model)__


## 1. Importing Libraries & Dataset
<a id="importing-libraries-dataset" class="anchor"></a>

## 1.1. Import Necessary Libraries
<a id="import-necessary-libraries" class="anchor"></a>

# 1. Importing Libraries & Dataset

## 1.1 Import Necessary Libraries

<span style="color:red"><b>>>>>>>> remote</b></span>

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb



from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)

# Feature selection
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# Sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Import labelencoder
from sklearn.preprocessing import LabelEncoder

# Import helper_functions
import sys
sys.path.append('../helper_functions')
from helper_functions import *

# For the dashboard
import joblib



## 1.2. Load the Dataset
<a id="load-the-dataset" class="anchor"></a>
[Back to ToC](#toc)<br>

In [2]:
# You can download the data in the source that is linked above the table of contents

# Read in the data
X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)
X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)

y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

## 2. Prepare the Dataset
<a id="prepare-the-dataset" class="anchor"></a>
[Back to ToC](#toc)<br>

## 2.1. Encode the Target Variable
<a id="encode-the-target-variable" class="anchor"></a>
[Back to ToC](#toc)<br>

> We decided to label encode the target variable in the following way:
>    | Claim Injury Type   | Encoded Value |
>    |---------------------|---------------|
>    | 4. TEMPORARY        | 3             |
>    | 2. NON-COMP         | 1             |
>    | 5. PPD SCH LOSS     | 4             |
>    | 3. MED ONLY         | 2             |
>    | 6. PPD NSL          | 5             |
>    | 1. CANCELLED        | 0             |
>    | 8. DEATH            | 7             |
>    | 7. PTD              | 6             |

In [3]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

label_encoder = LabelEncoder()

# Encoding y_train and converting it to a Series
y_train_encoded = pd.Series(label_encoder.fit_transform(y_train.values.ravel()), index=y_train.index)

# Encoding y_val and converting it to a Series
y_val_encoded = pd.Series(label_encoder.fit_transform(y_val.values.ravel()), index=y_val.index)


In [4]:
# View the original classes and their encoded values
for original, encoded in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{original} -> {encoded}")


1. CANCELLED -> 0
2. NON-COMP -> 1
3. MED ONLY -> 2
4. TEMPORARY -> 3
5. PPD SCH LOSS -> 4
6. PPD NSL -> 5
7. PTD -> 6
8. DEATH -> 7


## 2.2. Non-encoded Target Variable as Series
<a id="non-encoded-target-variable-as-series" class="anchor"></a>
[Back to ToC](#toc)<br>

> We convert the target variable into a series so we can use it for the Categorical Feature Selection

In [5]:
y = y_train.squeeze()

## 2.3. Separate the Variables into Numerical and Categorical
<a id="separate-the-variables-into-numerical-and-categorical" class="anchor"></a>
[Back to ToC](#toc)<br>

> __Separate the variables into categorical and numerical__
>
> | Numerical Columns                                     | Categorical Columns                          |
> |-------------------------------------------------------|----------------------------------------------|
> | Age at Injury                                         | Alternative Dispute Resolution_binary        |
> | IME-4 Count                                           | COVID-19 Indicator_binary                   |
> | Number of Dependents                                  | Attorney/Representative_binary              |
> | Days_between_Assembly Date_Accident Date_log          | Carrier Type_Self-insured Private Entity     |
> | Days_between_C-2 Date_Imputed_Accident Date_log               | Carrier Type_Self-insured Public Entity      |
> | Average Weekly Wage_Imputed_log                       | Carrier Type_Special Funds                  |
> | Industry Code_encoded_5. PPD SCH LOSS                | Carrier Type_State Insurance Fund           |
> | Industry Code_encoded_2. NON-COMP                    | C-3 Date_nabinary                           |
> | Industry Code_encoded_3. MED ONLY                    | First Hearing Date_nabinary                |
> | Industry Code_encoded_4. TEMPORARY                   | Accident Date_Season_Spring                |
> | Industry Code_encoded_1. CANCELLED                   | Accident Date_Season_Summer                |
> | Industry Code_encoded_8. DEATH                       | Accident Date_Season_Winter                |
> | Industry Code_encoded_6. PPD NSL                     | Average Weekly Wage_nabinary               |
> | Industry Code_encoded_7. PTD                         |                                              |
> | WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS    |                                              |
> | WCIO Cause of Injury Code_encoded_2. NON-COMP        |                                              |
> | WCIO Cause of Injury Code_encoded_3. MED ONLY        |                                              |
> | WCIO Cause of Injury Code_encoded_4. TEMPORARY       |                                              |
> | WCIO Cause of Injury Code_encoded_1. CANCELLED       |                                              |
> | WCIO Cause of Injury Code_encoded_8. DEATH           |                                              |
> | WCIO Cause of Injury Code_encoded_6. PPD NSL         |                                              |
> | WCIO Cause of Injury Code_encoded_7. PTD             |                                              |
> | WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS   |                                              |
> | WCIO Nature of Injury Code_encoded_2. NON-COMP       |                                              |
> | WCIO Nature of Injury Code_encoded_3. MED ONLY       |                                              |
> | WCIO Nature of Injury Code_encoded_4. TEMPORARY      |                                              |
> | WCIO Nature of Injury Code_encoded_1. CANCELLED      |                                              |
> | WCIO Nature of Injury Code_encoded_8. DEATH          |                                              |
> | WCIO Nature of Injury Code_encoded_6. PPD NSL        |                                              |
> | WCIO Nature of Injury Code_encoded_7. PTD            |                                              |
> | WCIO Part Of Body Code_encoded_5. PPD SCH LOSS       |                                              |
> | WCIO Part Of Body Code_encoded_2. NON-COMP           |                                              |
> | WCIO Part Of Body Code_encoded_3. MED ONLY           |                                              |
> | WCIO Part Of Body Code_encoded_4. TEMPORARY          |                                              |
> | WCIO Part Of Body Code_encoded_1. CANCELLED          |                                              |
> | WCIO Part Of Body Code_encoded_8. DEATH              |                                              |
> | WCIO Part Of Body Code_encoded_6. PPD NSL            |                                              |
> | WCIO Part Of Body Code_encoded_7. PTD                |                                              |
> | Accident Datemonth                                   |                                              |


In [6]:
num_columns = ['Age at Injury', 'IME-4 Count', 'Number of Dependents',
               'Days_between_Assembly Date_Accident Date_log',
               'Days_between_C-2 Date_Imputed_Accident Date_log', 'Average Weekly Wage_Imputed_log',
               'Industry Code_encoded_5. PPD SCH LOSS',
               'Industry Code_encoded_2. NON-COMP',
               'Industry Code_encoded_3. MED ONLY',
               'Industry Code_encoded_4. TEMPORARY',
               'Industry Code_encoded_1. CANCELLED', 'Industry Code_encoded_8. DEATH',
               'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
               'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Cause of Injury Code_encoded_2. NON-COMP',
               'WCIO Cause of Injury Code_encoded_3. MED ONLY',
               'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
               'WCIO Cause of Injury Code_encoded_1. CANCELLED',
               'WCIO Cause of Injury Code_encoded_8. DEATH',
               'WCIO Cause of Injury Code_encoded_6. PPD NSL',
               'WCIO Cause of Injury Code_encoded_7. PTD',
               'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Nature of Injury Code_encoded_2. NON-COMP',
               'WCIO Nature of Injury Code_encoded_3. MED ONLY',
               'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
               'WCIO Nature of Injury Code_encoded_1. CANCELLED',
               'WCIO Nature of Injury Code_encoded_8. DEATH',
               'WCIO Nature of Injury Code_encoded_6. PPD NSL',
               'WCIO Nature of Injury Code_encoded_7. PTD',
               'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
               'WCIO Part Of Body Code_encoded_2. NON-COMP',
               'WCIO Part Of Body Code_encoded_3. MED ONLY',
               'WCIO Part Of Body Code_encoded_4. TEMPORARY',
               'WCIO Part Of Body Code_encoded_1. CANCELLED',
               'WCIO Part Of Body Code_encoded_8. DEATH',
               'WCIO Part Of Body Code_encoded_6. PPD NSL',
               'WCIO Part Of Body Code_encoded_7. PTD',
               'Accident Datemonth']
            #    'Industry Code_freq', 'WCIO Cause of Injury Code_freq',
            #    'WCIO Nature of Injury Code_freq', 'WCIO Part Of Body Code_freq', 
            #    'Carrier Type_freq', 'Carrier Name_freq'

cat_columns = ['Alternative Dispute Resolution_binary', 
               'COVID-19 Indicator_binary',
               'Attorney/Representative_binary',
               'Carrier Type_Self-insured Private Entity',
               'Carrier Type_Self-insured Public Entity', 
               'Carrier Type_Special Funds',
               'Carrier Type_State Insurance Fund', 
               'C-3 Date_nabinary',
               'First Hearing Date_nabinary', 
               'Accident Date_Season_Spring',
               'Accident Date_Season_Summer', 
               'Accident Date_Season_Winter',
               'Average Weekly Wage_nabinary']


# Create subsets
X_train_num = X_train_encoded[num_columns]
X_train_cat = X_train_encoded[cat_columns]

X_val_num = X_val_encoded[num_columns]
X_val_cat = X_val_encoded[cat_columns]

X_test_num=X_test_encoded[num_columns]
X_test_cat=X_test_encoded[cat_columns]

## 3. Data Scaling
<a id="data-scaling" class="anchor"></a>
[Back to ToC](#toc)<br>

## 3.1. Normalization
<a id="normalization" class="anchor"></a>
[Back to ToC](#toc)<br>

> Not all models need the variables to be scaled.

In [7]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
X_train_num_scaled.describe().round(2)

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth
count,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00,396097.00
mean,0.41,0.04,0.50,0.30,0.30,0.33,0.29,0.50,0.43,0.42,0.08,0.20,0.55,0.26,0.18,0.43,0.51,0.37,0.04,0.01,0.16,0.01,0.17,0.52,0.58,0.45,0.04,0.01,0.28,0.00,0.23,0.34,0.61,0.54,0.04,0.02,0.15,0.03,0.50
std,0.20,0.09,0.33,0.16,0.17,0.40,0.34,0.31,0.19,0.25,0.07,0.14,0.19,0.17,0.13,0.18,0.13,0.13,0.15,0.02,0.11,0.02,0.11,0.19,0.15,0.20,0.15,0.03,0.18,0.02,0.19,0.24,0.15,0.19,0.15,0.07,0.16,0.06,0.32
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.25,0.00,0.17,0.19,0.18,0.00,0.06,0.17,0.26,0.26,0.04,0.12,0.40,0.16,0.13,0.31,0.47,0.31,0.01,0.00,0.06,0.00,0.06,0.41,0.55,0.34,0.02,0.00,0.09,0.00,0.04,0.14,0.57,0.37,0.01,0.00,0.03,0.00,0.18
50%,0.41,0.00,0.50,0.27,0.26,0.00,0.12,0.54,0.44,0.46,0.05,0.15,0.63,0.28,0.17,0.38,0.54,0.38,0.02,0.00,0.16,0.01,0.19,0.53,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.25,0.61,0.58,0.02,0.00,0.10,0.01,0.55
75%,0.58,0.00,0.83,0.38,0.40,0.79,0.33,0.74,0.63,0.55,0.10,0.23,0.67,0.30,0.21,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [8]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)
X_val_num_scaled.describe().round(2)

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth
count,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00,172200.00
mean,0.41,0.04,0.50,0.31,0.30,0.33,0.29,0.51,0.42,0.42,0.08,0.19,0.55,0.26,0.18,0.43,0.50,0.37,0.05,0.01,0.16,0.01,0.17,0.52,0.57,0.45,0.05,0.01,0.28,0.00,0.23,0.34,0.61,0.54,0.05,0.02,0.15,0.03,0.50
std,0.20,0.10,0.33,0.18,0.17,0.40,0.34,0.31,0.20,0.25,0.07,0.14,0.19,0.17,0.13,0.18,0.13,0.13,0.16,0.02,0.11,0.02,0.12,0.19,0.15,0.20,0.16,0.03,0.18,0.02,0.19,0.24,0.16,0.19,0.16,0.07,0.16,0.06,0.31
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.25,0.00,0.17,0.19,0.18,0.00,0.06,0.17,0.26,0.26,0.04,0.12,0.40,0.16,0.13,0.31,0.47,0.31,0.01,0.00,0.05,0.00,0.06,0.41,0.55,0.34,0.02,0.00,0.09,0.00,0.04,0.14,0.55,0.37,0.01,0.00,0.03,0.00,0.18
50%,0.41,0.00,0.50,0.27,0.26,0.00,0.12,0.54,0.44,0.46,0.05,0.15,0.63,0.28,0.17,0.38,0.54,0.38,0.02,0.00,0.16,0.01,0.19,0.53,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.26,0.61,0.58,0.02,0.00,0.10,0.01,0.55
75%,0.58,0.00,0.83,0.38,0.41,0.79,0.33,0.74,0.61,0.55,0.10,0.23,0.67,0.30,0.21,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73
max,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [9]:
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)
X_test_num_scaled.describe().round(2)

,Age at Injury,IME-4 Count,Number of Dependents,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,Average Weekly Wage_Imputed_log,Industry Code_encoded_5. PPD SCH LOSS,Industry Code_encoded_2. NON-COMP,Industry Code_encoded_3. MED ONLY,Industry Code_encoded_4. TEMPORARY,Industry Code_encoded_1. CANCELLED,Industry Code_encoded_8. DEATH,Industry Code_encoded_6. PPD NSL,Industry Code_encoded_7. PTD,WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS,WCIO Cause of Injury Code_encoded_2. NON-COMP,WCIO Cause of Injury Code_encoded_3. MED ONLY,WCIO Cause of Injury Code_encoded_4. TEMPORARY,WCIO Cause of Injury Code_encoded_1. CANCELLED,WCIO Cause of Injury Code_encoded_8. DEATH,WCIO Cause of Injury Code_encoded_6. PPD NSL,WCIO Cause of Injury Code_encoded_7. PTD,WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS,WCIO Nature of Injury Code_encoded_2. NON-COMP,WCIO Nature of Injury Code_encoded_3. MED ONLY,WCIO Nature of Injury Code_encoded_4. TEMPORARY,WCIO Nature of Injury Code_encoded_1. CANCELLED,WCIO Nature of Injury Code_encoded_8. DEATH,WCIO Nature of Injury Code_encoded_6. PPD NSL,WCIO Nature of Injury Code_encoded_7. PTD,WCIO Part Of Body Code_encoded_5. PPD SCH LOSS,WCIO Part Of Body Code_encoded_2. NON-COMP,WCIO Part Of Body Code_encoded_3. MED ONLY,WCIO Part Of Body Code_encoded_4. TEMPORARY,WCIO Part Of Body Code_encoded_1. CANCELLED,WCIO Part Of Body Code_encoded_8. DEATH,WCIO Part Of Body Code_encoded_6. PPD NSL,WCIO Part Of Body Code_encoded_7. PTD,Accident Datemonth
count,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00,387975.00
mean,0.40,0.01,0.50,0.29,0.28,0.15,0.26,0.54,0.43,0.40,0.07,0.19,0.54,0.25,0.18,0.44,0.51,0.36,0.04,0.00,0.16,0.01,0.17,0.53,0.58,0.43,0.04,0.01,0.28,0.00,0.23,0.34,0.63,0.54,0.04,0.02,0.15,0.02,0.43
std,0.20,0.04,0.33,0.17,0.16,0.32,0.32,0.32,0.19,0.26,0.07,0.13,0.19,0.17,0.12,0.18,0.13,0.14,0.15,0.02,0.11,0.02,0.11,0.19,0.15,0.20,0.15,0.03,0.17,0.02,0.18,0.24,0.15,0.19,0.15,0.05,0.15,0.05,0.32
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.23,0.00,0.17,0.16,0.16,0.00,0.06,0.17,0.26,0.26,0.04,0.08,0.40,0.15,0.13,0.31,0.48,0.31,0.01,0.00,0.06,0.00,0.06,0.41,0.56,0.34,0.02,0.00,0.17,0.00,0.05,0.12,0.57,0.37,0.01,0.00,0.03,0.00,0.18
50%,0.39,0.00,0.50,0.24,0.25,0.00,0.11,0.62,0.44,0.40,0.05,0.15,0.58,0.24,0.17,0.38,0.54,0.37,0.02,0.00,0.18,0.01,0.19,0.55,0.60,0.44,0.02,0.00,0.26,0.00,0.22,0.25,0.63,0.59,0.02,0.00,0.10,0.01,0.36
75%,0.57,0.00,0.83,0.36,0.37,0.00,0.33,0.80,0.63,0.48,0.10,0.23,0.67,0.30,0.20,0.52,0.56,0.45,0.02,0.00,0.21,0.01,0.25,0.56,0.66,0.57,0.02,0.00,0.44,0.00,0.31,0.54,0.72,0.68,0.03,0.03,0.24,0.03,0.73
max,1.00,0.90,1.00,1.05,1.02,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


## 3.2. Standardization
<a id="standardization" class="anchor"></a>
[Back to ToC](#toc)<br>

> We can use both, if we're using algorithms that benefit from using both.  (e.g., neural networks may benefit from normalization, while models like logistic regression may perform better with standardized data).

In [10]:
# from sklearn.preprocessing import StandardScaler

# std_scaler = StandardScaler().fit(X_train_num)
# X_train_scaled_std= scaler.transform(X_train_num)
# X_train_scaled_std = pd.DataFrame(X_train_scaled_std, columns = X_train_num.columns).set_index(X_train_encoded.index)

# X_val_scaled_std = scaler.transform(X_val_num)
# X_val_scaled_std = pd.DataFrame(X_val_scaled_std, columns = X_val_num.columns).set_index(X_val_encoded.index)


## 4. Feature Selection
<a id="feature-selection" class="anchor"></a>
[Back to ToC](#toc)<br>

## 4.1. Categorical: Filter Methods
<a id="categorical-filter-methods" class="anchor"></a>
[Back to ToC](#toc)<br>

## 4.1.1. Chi square and Cramer's V
<a id="chi-square-and-cramers-v" class="anchor"></a>
[Back to ToC](#toc)<br>

In [11]:
# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat, y)

# Create DataFrame for Chi-square results
chi2_results = pd.DataFrame({
    'Column': X_train_cat.columns,
    'Chi2': chi2_values.round(5),
    'p-value': p_values.round(5)
})

# Calculate Cramér's V for binary target
cramers_v_values = []
for var in X_train_cat.columns:
    v = cramers_v(X_train_cat[var], y)
    cramers_v_values.append(v)

# Add Cramér's V to DataFrame
chi2_results['Cramér\'s V'] = cramers_v_values

# Filter important features
chi2_important_features = chi2_results[(chi2_results['p-value'] < 0.05) & (chi2_results['Cramér\'s V'] >= 0.1)]

list_features_chi2_cramer = chi2_important_features['Column'].values

print(chi2_important_features)

                                     Column          Chi2  p-value  Cramér's V
2            Attorney/Representative_binary  102697.45907      0.0    0.614944
4   Carrier Type_Self-insured Public Entity    5842.39801      0.0    0.137058
6         Carrier Type_State Insurance Fund    4649.12143      0.0    0.120717
7                         C-3 Date_nabinary   29753.82707      0.0    0.479025
8               First Hearing Date_nabinary   30991.88712      0.0    0.545735
12             Average Weekly Wage_nabinary  110326.13914      0.0    0.872155


## 4.1.2. Mutual Information
<a id="mutual-information" class="anchor"></a>
[Back to ToC](#toc)<br>

In [12]:
# Since it's categorical features, we should include discrete_features=True
mi = mutual_info_classif(X_train_cat, y,discrete_features=True)

# Get a dataframe with the results
mi_results = pd.DataFrame({
    'Feature': X_train_cat.columns,
    'Mutual Information': mi
})

# Sort by mutual information score
mi_results = mi_results.sort_values(by='Mutual Information', ascending=False)
mi_results


,Feature,Mutual Information
12,Average Weekly Wage_nabinary,0.469901
2,Attorney/Representative_binary,0.202553
8,First Hearing Date_nabinary,0.159525
7,C-3 Date_nabinary,0.117598
4,Carrier Type_Self-insured Public Entity,0.008491
6,Carrier Type_State Insurance Fund,0.007216
1,COVID-19 Indicator_binary,0.006363
0,Alternative Dispute Resolution_binary,0.002981
3,Carrier Type_Self-insured Private Entity,0.001237
11,Accident Date_Season_Winter,0.000167


> A threshold of 0.01 for mutual information was selected to adopt a conservative approach in feature filtering. This threshold ensures that only features with a small but non-zero relationship to the target are retained, avoiding the exclusion of potentially informative features.

In [13]:
mi_important_features = mi_results[mi_results['Mutual Information'] > 0.01]

list_features_mi = mi_results[mi_results['Mutual Information'] > 0.01]['Feature'].values

print(mi_important_features)

                           Feature  Mutual Information
12    Average Weekly Wage_nabinary            0.469901
2   Attorney/Representative_binary            0.202553
8      First Hearing Date_nabinary            0.159525
7                C-3 Date_nabinary            0.117598


## 4.1.3. Filtered Categoricals
<a id="filtered-categoricals" class="anchor"></a>
[Back to ToC](#toc)<br>

> Keep the features that are chosen by either one of the methods.
> Which are:
> * 'Carrier Type_Self-insured Public Entity'
> * 'Carrier Type_State Insurance Fund'
> * 'First Hearing Date_nabinary'
> * 'Attorney/Representative_binary'
> * 'Average Weekly Wage_nabinary'
> * 'C-3 Date_nabinary'

In [14]:
categoricals_to_keep = list(set(list_features_chi2_cramer) | set(list_features_mi))

In [15]:
# Keep only important features
X_train_cat_filtered = X_train_cat[categoricals_to_keep]
X_val_cat_filtered = X_val_cat[categoricals_to_keep]
X_test_cat_filtered = X_test_cat[categoricals_to_keep]

# Verify the remaining columns
print("Remaining features in X_train_cat:", X_train_cat_filtered.columns.tolist())


Remaining features in X_train_cat: ['Attorney/Representative_binary', 'Carrier Type_Self-insured Public Entity', 'Carrier Type_State Insurance Fund', 'First Hearing Date_nabinary', 'Average Weekly Wage_nabinary', 'C-3 Date_nabinary']


## 4.2. Numerical: Filter Methods
<a id="numerical-filter-methods" class="anchor"></a>
[Back to ToC](#toc)<br>

## 4.2.1. Univariate Variance
<a id="univariate-variance" class="anchor"></a>
[Back to ToC](#toc)<br>

In [16]:
X_train_num_scaled.var().sort_values(ascending=False)

Average Weekly Wage_Imputed_log                       0.158004
Industry Code_encoded_5. PPD SCH LOSS                 0.114255
Number of Dependents                                  0.111300
Accident Datemonth                                    0.099293
Industry Code_encoded_2. NON-COMP                     0.097401
Industry Code_encoded_4. TEMPORARY                    0.064422
WCIO Part Of Body Code_encoded_2. NON-COMP            0.057635
WCIO Nature of Injury Code_encoded_4. TEMPORARY       0.039304
Age at Injury                                         0.039004
Industry Code_encoded_3. MED ONLY                     0.037838
WCIO Part Of Body Code_encoded_5. PPD SCH LOSS        0.037588
WCIO Nature of Injury Code_encoded_2. NON-COMP        0.037585
WCIO Part Of Body Code_encoded_4. TEMPORARY           0.037144
Industry Code_encoded_6. PPD NSL                      0.034938
WCIO Cause of Injury Code_encoded_2. NON-COMP         0.032058
WCIO Nature of Injury Code_encoded_6. PPD NSL         0

> The variance analysis is performed on the numerical features in the dataset. Since none of the numerical features have zero variance, all numerical features will be retained for now.

## 4.2.2. Spearman Correlation
<a id="spearman-correlation" class="anchor"></a>
[Back to ToC](#toc)<br>

> We set the threshold for Spearman correlation at |0.9| (above 0.9 or below -0.9) to identify and remove highly correlated features. So we can avoid multicollinearity in the model.

In [17]:
# Calculate Spearman correlation matrix
cor_spearman = X_train_num_scaled.corr(method='spearman')

correlation_pairs = cor_spearman.unstack().reset_index()

correlation_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# Filter the table for correlations > 0.9 or < -0.9 and exclude self-correlations
strong_correlations = correlation_pairs[
    ((correlation_pairs['Correlation'] > 0.9) | (correlation_pairs['Correlation'] < -0.9)) &
    (correlation_pairs['Feature_1'] != correlation_pairs['Feature_2'])
]

# Show unique pairs only
strong_correlations = strong_correlations.sort_values(by='Correlation', ascending=False).drop_duplicates(subset=['Correlation'])

# Display the table
strong_correlations


,Feature_1,Feature_2,Correlation
121,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Imputed_Accident Date_log,0.954048


## 4.2.3. Mutual Information
<a id="mutual-information-2" class="anchor"></a>
[Back to ToC](#toc)<br>

In [18]:
# # Compute mutual information between categorical features and target
# mutual_info = mutual_info_classif(X_train_num_scaled, y_train_encoded)

# # Display features sorted by mutual information
# mi_results = pd.DataFrame({
#     'Feature': X_train_num_scaled.columns,
#     'Mutual Information': mutual_info
# }).sort_values(by='Mutual Information', ascending=False)

# mi_results

In [19]:
mi_important_features_numerical = mi_results[mi_results['Mutual Information'] < 0.01]

non_important_features_mi = mi_results[mi_results['Mutual Information'] < 0.01]['Feature'].values

print(mi_important_features_numerical)

                                     Feature  Mutual Information
4    Carrier Type_Self-insured Public Entity            0.008491
6          Carrier Type_State Insurance Fund            0.007216
1                  COVID-19 Indicator_binary            0.006363
0      Alternative Dispute Resolution_binary            0.002981
3   Carrier Type_Self-insured Private Entity            0.001237
11               Accident Date_Season_Winter            0.000167
5                 Carrier Type_Special Funds            0.000148
9                Accident Date_Season_Spring            0.000074
10               Accident Date_Season_Summer            0.000057


In [20]:
# Takes a long time to run Mutual Information in Numerical again, so we will keep it here:
non_important_features_mi = ['Number of Dependents', 'Accident Datemonth']

## 4.2.4. Filtered Numerical
<a id="filtered-numerical" class="anchor"></a>
[Back to ToC](#toc)<br>

> Drop the features that are considered not important by Mutual Information and drop one of the features that has high correlation.
> We drop:
> * Days_between_C-2 Date_Imputed_Accident Date_log
> * Number of Dependents
> * Accident Datemonth

In [21]:
# List of features to drop from X_train_num_scaled
num_features_to_drop = ['Days_between_C-2 Date_Imputed_Accident Date_log'] + list(non_important_features_mi)

# Drop features
X_train_num_scaled_filtered = X_train_num_scaled.drop(columns=num_features_to_drop)
X_val_num_scaled_filtered = X_val_num_scaled.drop(columns=num_features_to_drop)
X_test_num_scaled_filtered = X_test_num_scaled.drop(columns=num_features_to_drop)

# Verify the remaining columns
print("Remaining features in X_train_num_scaled:", X_train_num_scaled_filtered.columns.tolist())


Remaining features in X_train_num_scaled: ['Age at Injury', 'IME-4 Count', 'Days_between_Assembly Date_Accident Date_log', 'Average Weekly Wage_Imputed_log', 'Industry Code_encoded_5. PPD SCH LOSS', 'Industry Code_encoded_2. NON-COMP', 'Industry Code_encoded_3. MED ONLY', 'Industry Code_encoded_4. TEMPORARY', 'Industry Code_encoded_1. CANCELLED', 'Industry Code_encoded_8. DEATH', 'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD', 'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Cause of Injury Code_encoded_2. NON-COMP', 'WCIO Cause of Injury Code_encoded_3. MED ONLY', 'WCIO Cause of Injury Code_encoded_4. TEMPORARY', 'WCIO Cause of Injury Code_encoded_1. CANCELLED', 'WCIO Cause of Injury Code_encoded_8. DEATH', 'WCIO Cause of Injury Code_encoded_6. PPD NSL', 'WCIO Cause of Injury Code_encoded_7. PTD', 'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS', 'WCIO Nature of Injury Code_encoded_2. NON-COMP', 'WCIO Nature of Injury Code_encoded_3. MED ONLY', 'WCIO 

## 4.3. Combine the Filtered Datasets
<a id="combine-the-filtered-datasets" class="anchor"></a>
[Back to ToC](#toc)<br>

> After filtering we should combine them into X_train X_val and X_test

In [22]:
# Combine the filtered datasets
X_train = pd.concat([X_train_cat_filtered, X_train_num_scaled_filtered], axis=1)
X_val =  pd.concat([X_val_cat_filtered, X_val_num_scaled_filtered], axis=1)
X_test =  pd.concat([X_test_cat_filtered, X_test_num_scaled_filtered], axis=1)


# Verify the shape of the combined dataset
print("Shape of combined X_train:", X_train.shape)


Shape of combined X_train: (396097, 42)


> We move on to the next stage of feature selection with 42 features.

## 4.4. Feature Selection on the Combined Dataset
<a id="feature-selection-on-the-combined-dataset" class="anchor"></a>
[Back to ToC](#toc)<br>

## 4.4.1. Random Forest Important Features
<a id="random-forest-important-features" class="anchor"></a>
[Back to ToC](#toc)<br>

> To identify the most important features for the model, we decided to use Random Forest as it provides a built-in feature importance. It also ranks features based on their contribution to reducing the impurity in decision trees.

In [23]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=47)

# Fit the model on X_train and y_train_encoded
rf_model.fit(X_train, y_train_encoded)

# Get feature importances
importances = rf_model.feature_importances_

# Sort feature importances in descending order
important_indices = importances.argsort()[::-1]

# Print the feature importances
print("Feature importances:", importances)
print("Sorted feature indices:", important_indices)


Feature importances: [0.04952505 0.00608349 0.0077917  0.03077846 0.15519269 0.02185146
 0.08060892 0.0491559  0.07959568 0.17280232 0.01284866 0.00896107
 0.0111146  0.00947726 0.00867778 0.0087109  0.00878062 0.00834176
 0.01566412 0.01383309 0.01543924 0.01426205 0.0144408  0.01088779
 0.0140902  0.0103554  0.00864208 0.01087441 0.00833092 0.01160431
 0.00814869 0.00552761 0.00684021 0.00592738 0.01600317 0.01071731
 0.0131757  0.01286008 0.01183201 0.00913848 0.01229937 0.00880727]
Sorted feature indices: [ 9  4  6  8  0  7  3  5 34 18 20 22 21 24 19 36 37 10 40 38 29 12 23 27
 35 25 13 39 11 41 16 15 14 26 17 28 30  2 32  1 33 31]


> To reduce the dimensionality of the model while retaining the most important features, we used the cumulative feature importance from the Random Forest model. By calculating the cumulative sum of feature importances, we identified the smallest subset of features that collectively explain 90% of the total importance.

In [24]:
# Cumulative sum of feature importance
cumulative_importance = np.cumsum(importances[important_indices])

# Set threshold of 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The the number of features
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

random_forest_features = X_train.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {random_forest_features}")


Number of features explaining 90% of cumulative importance: 30
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_Imputed_log', 'Average Weekly Wage_nabinary',
       'Age at Injury', 'Days_between_Assembly Date_Accident Date_log',
       'Attorney/Representative_binary', 'IME-4 Count',
       'First Hearing Date_nabinary', 'C-3 Date_nabinary',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_2. NON-COMP',
       'WCIO Part Of Body Code_encoded_3. MED ONLY',
       'WCIO Part Of Body Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'WCIO Part Of Body Code_encoded_6. PPD NSL',
       'WC

## 4.4.2. XGBoost Important Features
<a id="xgboost-important-features" class="anchor"></a>
[Back to ToC](#toc)<br>

> We decided to use XGBoost and Random Forest for feature importance, given our large dataset. These models are well-suited for multiclass classification.

In [25]:
# Initialize the XGBoost Classifier
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=47)

# Fit the model on X_train and y_train_encoded
xgb_model.fit(X_train, y_train_encoded)

# Get feature importances
importances = xgb_model.feature_importances_

# Sort feature importances in descending order
important_indices = importances.argsort()[::-1]

# Print the feature importances
print("Feature importances:", importances)
print("Sorted feature indices:", important_indices)


Feature importances: [0.09888545 0.00902688 0.00772492 0.01603887 0.6627125  0.00490609
 0.00155897 0.01340491 0.00267976 0.04499499 0.01163969 0.00216043
 0.00231986 0.00246448 0.0024249  0.00183366 0.00250933 0.00137726
 0.00379072 0.00155391 0.01753235 0.00179809 0.00497774 0.00155375
 0.00155075 0.00138288 0.0042116  0.00563838 0.00795785 0.00647521
 0.01034574 0.00216845 0.00165787 0.00137871 0.01862336 0.00183377
 0.00580694 0.00208119 0.00168467 0.00273629 0.0033978  0.0011991 ]
Sorted feature indices: [ 4  0  9 34 20  3  7 10 30  1 28  2 29 36 27 22  5 26 18 40 39  8 16 13
 14 12 31 11 37 35 15 21 38 32  6 19 23 24 25 33 17 41]


In [26]:
# Cumulative sum of feature importance
cumulative_importance = np.cumsum(importances[important_indices])

# Set threshold of 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

xgboost_features = X_train.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {xgboost_features}")


Number of features explaining 90% of cumulative importance: 10
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_nabinary', 'Attorney/Representative_binary',
       'Average Weekly Wage_Imputed_log',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'First Hearing Date_nabinary', 'IME-4 Count',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_1. CANCELLED',
       'Carrier Type_Self-insured Public Entity'],
      dtype='object')


## 4.4.3. Lasso
<a id="lasso" class="anchor"></a>
[Back to ToC](#toc)<br>

> Lasso excludes features whose coefficients become zero, meaning we keep all features with non-zero coefficients, as they are contributing to the model's performance.

In [27]:


# check where the missing values are
X_train.isna().sum()


Attorney/Representative_binary                        0
Carrier Type_Self-insured Public Entity               0
Carrier Type_State Insurance Fund                     0
First Hearing Date_nabinary                           0
Average Weekly Wage_nabinary                          0
C-3 Date_nabinary                                     0
Age at Injury                                         0
IME-4 Count                                           0
Days_between_Assembly Date_Accident Date_log          0
Average Weekly Wage_Imputed_log                       0
Industry Code_encoded_5. PPD SCH LOSS                 0
Industry Code_encoded_2. NON-COMP                     0
Industry Code_encoded_3. MED ONLY                     0
Industry Code_encoded_4. TEMPORARY                    0
Industry Code_encoded_1. CANCELLED                    0
Industry Code_encoded_8. DEATH                        0
Industry Code_encoded_6. PPD NSL                      0
Industry Code_encoded_7. PTD                    

In [28]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5)  # Cross-validation to select optimal alpha
lasso.fit(X_train, y_train_encoded)
lasso_features = X_train.columns[lasso.coef_ != 0]
print("Selected features by Lasso:", lasso_features)

Selected features by Lasso: Index(['Attorney/Representative_binary',
       'Carrier Type_Self-insured Public Entity',
       'Carrier Type_State Insurance Fund', 'First Hearing Date_nabinary',
       'Average Weekly Wage_nabinary', 'C-3 Date_nabinary', 'Age at Injury',
       'IME-4 Count', 'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_2. NON-COMP',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
       'WCIO Nature of Injury Code_encoded_6. PPD NSL',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Part Of

## 4.4.4. Ridge
<a id="ridge" class="anchor"></a>
[Back to ToC](#toc)<br>

> We tested and got that the best threshold was 0.01.

In [31]:
import numpy as np

# Initialize Ridge regression model with cross-validation
ridge_model = RidgeCV(cv=5)

# Fit the model on the training data
ridge_model.fit(X_train, y_train_encoded)

# Get the feature coefficients
coefficients = ridge_model.coef_

# Set a threshold to select features (e.g., absolute coefficient > 0.01)
threshold = 0.01
ridge_features = X_train.columns[np.abs(coefficients) > threshold]

print(f"Selected features using Ridge regression: {ridge_features}")


Selected features using Ridge regression: Index(['Attorney/Representative_binary',
       'Carrier Type_Self-insured Public Entity',
       'First Hearing Date_nabinary', 'Average Weekly Wage_nabinary',
       'C-3 Date_nabinary', 'Age at Injury', 'IME-4 Count',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_1. CANCELLED',
       'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_8. DEATH',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code

## 4.4.5. Vote for the Best Features
<a id="vote-for-the-best-features" class="anchor"></a>
[Back to ToC](#toc)<br>

> Save the features selected here so we don't have to run it all again.

In [32]:
rf_set = ['Average Weekly Wage_Imputed_log', 'Average Weekly Wage_nabinary',
       'Age at Injury', 'Days_between_Assembly Date_Accident Date_log',
       'Attorney/Representative_binary', 'IME-4 Count',
       'First Hearing Date_nabinary', 'C-3 Date_nabinary',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'WCIO Cause of Injury Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
       'WCIO Part Of Body Code_encoded_3. MED ONLY',
       'WCIO Part Of Body Code_encoded_4. TEMPORARY',
       'WCIO Part Of Body Code_encoded_1. CANCELLED',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_3. MED ONLY',
       'WCIO Part Of Body Code_encoded_6. PPD NSL',
       'WCIO Part Of Body Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_8. DEATH',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
       'WCIO Nature of Injury Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_7. PTD',
       'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'WCIO Part Of Body Code_encoded_8. DEATH',
       'Industry Code_encoded_6. PPD NSL']

xgboost_set = ['Average Weekly Wage_nabinary', 'Attorney/Representative_binary',
       'Average Weekly Wage_Imputed_log',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'First Hearing Date_nabinary',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY', 'IME-4 Count',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_1. CANCELLED',
       'Carrier Type_Self-insured Public Entity']

lasso_set = ['Carrier Type_Self-insured Public Entity',
       'Carrier Type_State Insurance Fund', 'First Hearing Date_nabinary',
       'Attorney/Representative_binary', 'Average Weekly Wage_nabinary',
       'C-3 Date_nabinary', 'Age at Injury', 'IME-4 Count',
       'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_2. NON-COMP',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
       'WCIO Nature of Injury Code_encoded_6. PPD NSL',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Part Of Body Code_encoded_2. NON-COMP',
       'WCIO Part Of Body Code_encoded_3. MED ONLY']

ridge_set = ['Carrier Type_Self-insured Public Entity',
       'First Hearing Date_nabinary', 'Attorney/Representative_binary',
       'Average Weekly Wage_nabinary', 'C-3 Date_nabinary', 'Age at Injury',
       'IME-4 Count', 'Days_between_Assembly Date_Accident Date_log',
       'Average Weekly Wage_Imputed_log',
       'Industry Code_encoded_5. PPD SCH LOSS',
       'Industry Code_encoded_4. TEMPORARY',
       'Industry Code_encoded_1. CANCELLED',
       'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
       'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Cause of Injury Code_encoded_2. NON-COMP',
       'WCIO Cause of Injury Code_encoded_3. MED ONLY',
       'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
       'WCIO Cause of Injury Code_encoded_1. CANCELLED',
       'WCIO Cause of Injury Code_encoded_8. DEATH',
       'WCIO Cause of Injury Code_encoded_6. PPD NSL',
       'WCIO Cause of Injury Code_encoded_7. PTD',
       'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
       'WCIO Nature of Injury Code_encoded_2. NON-COMP',
       'WCIO Nature of Injury Code_encoded_3. MED ONLY',
       'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
       'WCIO Nature of Injury Code_encoded_1. CANCELLED',
       'WCIO Nature of Injury Code_encoded_8. DEATH',
       'WCIO Nature of Injury Code_encoded_6. PPD NSL',
       'WCIO Nature of Injury Code_encoded_7. PTD',
       'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
       'WCIO Part Of Body Code_encoded_2. NON-COMP',
       'WCIO Part Of Body Code_encoded_3. MED ONLY',
       'WCIO Part Of Body Code_encoded_4. TEMPORARY',
       'WCIO Part Of Body Code_encoded_1. CANCELLED',
       'WCIO Part Of Body Code_encoded_8. DEATH',
       'WCIO Part Of Body Code_encoded_7. PTD']



> VOTING: AT LEAST 2 OUT OF 3

In [33]:
# Convert feature sets to Python sets
rf_set = set(rf_set)
lasso_set = set(lasso_set)
ridge_set = set(ridge_set)
xgboost_set = set(xgboost_set)

# Find features selected by at least 3 out of the 4 methods
final_features_set = (
    (rf_set & lasso_set & ridge_set) |
    (rf_set & lasso_set & xgboost_set) |
    (rf_set & ridge_set & xgboost_set) |
    (lasso_set & ridge_set & xgboost_set)
)

# Convert to sorted list
final_features = sorted(final_features_set)

# Display the final feature set
print(f"Number of selected by at least 3 methods: {len(final_features)} features.")


Number of selected by at least 3 methods: 22 features.


> Show which ones were chosen and by which methods.

In [34]:

# Combine all feature sets to get the union of all features
all_features = sorted(rf_set | lasso_set | ridge_set | xgboost_set)

# Create a table of feature selection for each method
feature_table = pd.DataFrame({
    'Feature': all_features,
    'RandomForest': [True if feature in rf_set else False for feature in all_features],
    'Lasso': [True if feature in lasso_set else False for feature in all_features],
    'Ridge': [True if feature in ridge_set else False for feature in all_features],
    'XGBoost': [True if feature in xgboost_set else False for feature in all_features],
})

# Add a new column that indicates whether the feature was chosen by at least 3 methods
feature_table['Chosen by 3+ Models'] = feature_table[['RandomForest', 'Lasso', 'Ridge', 'XGBoost']].sum(axis=1) >= 3

# Define color formatting function for the last column only
def highlight_chosen(val):
    color = 'background-color: lightgreen' if val == True else ''
    return color

# Apply conditional formatting to the last column using map instead of applymap
styled_table = feature_table.style.map(highlight_chosen, subset=['Chosen by 3+ Models'])

# Display the styled table
styled_table

,Feature,RandomForest,Lasso,Ridge,XGBoost,Chosen by 3+ Models
0,Age at Injury,True,True,True,False,True
1,Attorney/Representative_binary,True,True,True,True,True
2,Average Weekly Wage_Imputed_log,True,True,True,True,True
3,Average Weekly Wage_nabinary,True,True,True,True,True
4,C-3 Date_nabinary,True,True,True,False,True
5,Carrier Type_Self-insured Public Entity,False,True,True,True,True
6,Carrier Type_State Insurance Fund,False,True,False,False,False
7,Days_between_Assembly Date_Accident Date_log,True,True,True,False,True
8,First Hearing Date_nabinary,True,True,True,True,True
9,IME-4 Count,True,True,True,True,True


In [35]:
X_train = X_train[final_features]
X_val = X_val[final_features]
X_test = X_test[final_features]

## 5. Find the Best Model
<a id="find-the-best-model" class="anchor"></a>
[Back to ToC](#toc)<br>

> We used the ones that were taugh in class but also tried XGBoost as we believe that it will give us good results.

In [36]:
# # Define the models
# models = {
#     "MLPClassifier": MLPClassifier(max_iter=500),
#     "RandomForest": RandomForestClassifier(),
#     "LogisticRegression": LogisticRegression(max_iter=500),
#     "KNeighborsClassifier": KNeighborsClassifier(),
#     "DecisionTree": DecisionTreeClassifier(),
#     "GaussianNB": GaussianNB(),
#     "RidgeClassifier": RidgeClassifier(),
#     "XGBoost": xgb.XGBClassifier()
# }

# # Dictionary to store evaluation results
# results = {}

# # Iterate over each model
# for model_name, model in models.items():
#     print(f"Training {model_name}...")
#     # Fit the model on the training data
#     model.fit(X_train, y_train_encoded)
    
#     # Predict on the validation data
#     y_pred = model.predict(X_val)
    
#     # Evaluate the model
#     accuracy = accuracy_score(y_val, y_pred)
#     f1_macro = f1_score(y_val_encoded, y_pred, average='macro') 
    
#     # Store results
#     results[model_name] = {
#         "accuracy": accuracy,
#         "f1_macro": f1_macro
#     }
    
#     # Print individual model results
#     print(f"{model_name} Accuracy: {accuracy:.4f}")
#     print(f"{model_name} Macro F1 Score: {f1_macro:.4f}")
#     print(classification_report(y_val_encoded, y_pred))  # Detailed report including precision, recall, and F1 score per class
#     print("-" * 50)

# # Display the summary of results
# print("\nModel Evaluation Results Summary:")
# for model_name, metrics in results.items():
#     print(f"{model_name}: Accuracy = {metrics['accuracy']:.4f}, Macro F1 Score = {metrics['f1_macro']:.4f}")


> As we can see, XGBoost gave us the best Macro F1 score of 0.4183.

## 6. Deal with Imbalanced Dataset
<a id="deal-with-imbalanced-dataset" class="anchor"></a>
[Back to ToC](#toc)<br>

> By looking at the distribution of the target below, and as we have seen before, the class '2.NON-COMP' (1) counts for 51% of the rows. While the class with the least amount of rows has only 0.02%.

In [37]:
# Assuming `y` is your target variable
class_distribution = y_train_encoded.value_counts()
class_percentages = y_train_encoded.value_counts(normalize=True) * 100

# Combine into a single DataFrame for better visualization
distribution_df = pd.DataFrame({
    'Class': class_distribution.index,
    'Count': class_distribution.values,
    'Percentage (%)': class_percentages.values
})
print(distribution_df)


   Class   Count  Percentage (%)
0      1  202245       51.059463
1      3  101242       25.559901
2      2   47591       12.014986
3      4   33513        8.460806
4      0    8191        2.067928
5      5    2924        0.738203
6      7     324        0.081798
7      6      67        0.016915


> To try and tackle this problem we use tried different undersampling ways. And came to the conclusion that this is the one that gave us a better score.

In [38]:
# Define undersampling strategy based on the actual class distribution
undersample_strategy = {
    1: 20224,
    3: 20248,
    2: 47591,
    4: 33513,
    0: 8191,
    5: 2924,
    7: 324,
    6:67
}

# Apply undersampling to majority classes
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train_encoded)


# Check the new class distribution after resampling
unique_classes_final, class_counts_final = np.unique(y_resampled, return_counts=True)
class_distribution_final = dict(zip(unique_classes_final, class_counts_final))
print("New Class Distribution after Resampling:", class_distribution_final)

New Class Distribution after Resampling: {0: 8191, 1: 20224, 2: 47591, 3: 20248, 4: 33513, 5: 2924, 6: 67, 7: 324}


> We also tried to use SMOTE and oversample, but it didn't yield any results.

In [39]:
# # Define SMOTE strategy for oversampling the minority classes
# smote = SMOTE(sampling_strategy={
#     7:807,

#     6: 670,   # Oversample class '5' to 3,000
# }, random_state=42)

# # Apply SMOTE to oversample minority classes
# X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

> Given that undersampling may discard valuable information, adding a weight to the undersampled majority class ensures that the remaining samples still have sufficient influence during training, compensating for the data reduction.

In [40]:
weights = np.ones(len(y_resampled))
weights

majority_class_label_1 = 1  
majority_class_label_3 = 3



weights[y_resampled == majority_class_label_1] = 10
weights[y_resampled == majority_class_label_3] = 5
weights

array([1., 1., 1., ..., 1., 1., 1.])

In [41]:

model = xgb.XGBClassifier(eval_metric="mlogloss")
model.fit(X_resampled, y_resampled, sample_weight=weights)

# 3. Make predictions
y_pred = model.predict(X_val)  # Assuming X_test is available


## 7. Final Model
<a id="final-model" class="anchor"></a>
[Back to ToC](#toc)<br>

> We apply the weights to XGBoost 

In [42]:
model = xgb.XGBClassifier(eval_metric="mlogloss")
model.fit(X_resampled, y_resampled, sample_weight=weights)

y_pred = model.predict(X_val) 

In [43]:
accuracy = accuracy_score(y_val_encoded, y_pred)
f1_macro = f1_score(y_val_encoded, y_pred, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_encoded, y_pred))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

# Display the summary of results
print("\nModel Evaluation Results Summary:")
print(f"XGBClassifier: Accuracy = {accuracy:.4f}, Macro F1 Score = {f1_macro:.4f}")


XGBClassifier Accuracy: 0.7764
XGBClassifier Macro F1 Score: 0.4356
              precision    recall  f1-score   support

           0       0.53      0.48      0.50      3741
           1       0.85      0.96      0.90     87319
           2       0.41      0.11      0.17     20672
           3       0.73      0.83      0.77     44551
           4       0.63      0.62      0.63     14484
           5       0.13      0.01      0.01      1263
           6       0.00      0.00      0.00        29
           7       0.54      0.46      0.50       141

    accuracy                           0.78    172200
   macro avg       0.48      0.43      0.44    172200
weighted avg       0.74      0.78      0.74    172200

--------------------------------------------------

Model Evaluation Results Summary:
XGBClassifier: Accuracy = 0.7764, Macro F1 Score = 0.4356


In [44]:
# export to pickl
joblib.dump(model, '../dashboard_objects/model.joblib')

['../dashboard_objects/model.joblib']

### APPLY TO TEST

In [45]:
y_test_pred = model.predict(X_test)

In [46]:
# Analyze the distribution of predicted classes
predictions_df = pd.DataFrame(y_test_pred, columns=["Predicted Label"])
print("Predicted Class Distribution:")
print(predictions_df["Predicted Label"].value_counts(normalize=True))


Predicted Class Distribution:
Predicted Label
1    0.804044
3    0.125872
2    0.030698
4    0.019733
0    0.019352
7    0.000265
5    0.000036
Name: proportion, dtype: float64


In [47]:
X_previsao = X_test.copy()

In [48]:
X_previsao.loc[:, 'Predicted_Claim_Injury_Type'] = y_test_pred

In [49]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 3,
    '2. NON-COMP': 1,
    '5. PPD SCH LOSS': 4,
    '3. MED ONLY': 2,
    '6. PPD NSL': 5,
    '1. CANCELLED': 0,
    '8. DEATH': 7,
    '7. PTD': 6
}

# Reverse the mapping
reverse_claim_injury_type_mapping = {v: k for k, v in claim_injury_type_mapping.items()}

# Use the reversed mapping to get the original labels
X_previsao['Predicted_Claim_Injury_Type'] = X_previsao['Predicted_Claim_Injury_Type'].map(reverse_claim_injury_type_mapping)


In [50]:
X_previsao[['Predicted_Claim_Injury_Type']].to_csv('test_predictions.csv')
